In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

In [112]:
df = pd.read_csv("merged_table_23-24_with_null.csv", encoding="ISO-8859-1")

In [113]:
df['time'] = pd.to_datetime(df['time'], format='%m/%d/%y %H:%M')

In [114]:
# Iterate over the columns with object data type and apply the transformations
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].str.replace(r'\s+', '', regex=True)
    df[column] = df[column].replace('', np.nan)
    df[column] = df[column].astype(float)

In [115]:
print(df.isnull().sum())

time                   0
ni_in               4803
fe_in               4802
sio2_in             4802
cao_in              4802
mgo_in              4802
al2o3_in            4802
fe_ni               4803
s_m                 4802
bc                  4802
loi_in              8782
mc_kilnfeed            1
fc_coal             6624
gcv_coal            6624
voltage_pry          779
voltage_sec          510
current_pry          750
current_sec1         503
current_sec2         503
current_sec3         503
load                 498
power_factor         811
realisasi_beban      498
rpm                  745
kg_tco              5036
charge_kiln          256
tdo                    1
pry_p                256
sec_p                256
pry_v                256
sec_v                256
total_coal           284
total_fuel          4934
a_f_ratio           2100
reductor_consume     743
t_tic162            1112
t_tic163            1112
metal_temp          1179
ni_met              1177
c_met               1177


In [116]:
# Define the time ranges
start_range1 = pd.to_datetime('2024-07-31 01:00 AM')
end_range1 = pd.to_datetime('2024-07-31 03:00 PM')
start_range2 = pd.to_datetime('2024-08-01 12:00 AM')
end_range2 = pd.to_datetime('2024-11-12 11:00 PM')

# Filter the DataFrame to include only the specified time ranges
intersection_df = df[((df['time'] >= start_range1) & (df['time'] <= end_range1)) | ((df['time'] >= start_range2) & (df['time'] <= end_range2))]

# Display the filtered DataFrame
print(intersection_df.shape)

(2511, 48)


In [117]:
# Save the filtered data to a new CSV file (optional)
intersection_df.to_csv('filtered_intersection_df.csv', index=False)

In [118]:
print(intersection_df.isnull().sum())

time                   0
ni_in                  0
fe_in                  0
sio2_in                0
cao_in                 0
mgo_in                 0
al2o3_in               0
fe_ni                  0
s_m                    0
bc                     0
loi_in              2511
mc_kilnfeed            1
fc_coal             1815
gcv_coal            1815
voltage_pry            1
voltage_sec            1
current_pry            2
current_sec1           1
current_sec2           1
current_sec3           1
load                   0
power_factor           3
realisasi_beban        0
rpm                    0
kg_tco                23
charge_kiln            8
tdo                    1
pry_p                  8
sec_p                  8
pry_v                  8
sec_v                  8
total_coal             8
total_fuel             8
a_f_ratio            124
reductor_consume       0
t_tic162               0
t_tic163               0
metal_temp             3
ni_met                 1
c_met                  1


In [119]:
input_cols = intersection_df.columns[intersection_df.columns.get_loc('ni_in'):intersection_df.columns.get_loc('t_tic163') + 1]
output_cols = intersection_df.columns[intersection_df.columns.get_loc('metal_temp'):intersection_df.columns.get_loc('loi_kalsin') + 1]
smelting_cols = intersection_df.columns[intersection_df.columns.get_loc('voltage_pry'):intersection_df.columns.get_loc('realisasi_beban') + 1]

In [120]:
intersection_df[output_cols] = intersection_df[output_cols].shift(-16)
intersection_df = intersection_df.iloc[:-16]

# Reset the index
intersection_df.reset_index(drop=True, inplace=True)

C:\Users\rianr\AppData\Local\Temp\ipykernel_9176\3846504361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_df[output_cols] = intersection_df[output_cols].shift(-16)


In [121]:
filtered_intersection_df = intersection_df.drop(columns=['loi_in', 'fe_ni', 'sio2_in', 'mgo_in', 'voltage_pry', 'voltage_sec'])
input_cols = input_cols.drop(['loi_in', 'fe_ni', 'sio2_in', 'mgo_in', 'voltage_pry', 'voltage_sec'])

In [122]:
print(output_cols)

Index(['metal_temp', 'ni_met', 'c_met', 'si_met', 'fe_met', 's_met', 'ni_slag',
       'fe_slag', 't_kalsin', 'pic_161', 'loi_kalsin'],
      dtype='object')


In [123]:
print(filtered_intersection_df.isnull().sum())

time                   0
ni_in                  0
fe_in                  0
cao_in                 0
al2o3_in               0
s_m                    0
bc                     0
mc_kilnfeed            1
fc_coal             1815
gcv_coal            1815
current_pry            2
current_sec1           1
current_sec2           1
current_sec3           1
load                   0
power_factor           3
realisasi_beban        0
rpm                    0
kg_tco                22
charge_kiln            8
tdo                    1
pry_p                  8
sec_p                  8
pry_v                  8
sec_v                  8
total_coal             8
total_fuel             8
a_f_ratio            123
reductor_consume       0
t_tic162               0
t_tic163               0
metal_temp             3
ni_met                 1
c_met                  1
si_met                 1
fe_met                 1
s_met                  1
ni_slag                3
fe_slag                3
t_kalsin               0


In [124]:
def interpolate(df, column):
    # Convert the index to numerical values for interpolation
    x = np.arange(len(df))
    y = df[column].values

    # Identify the indices of the non-null values
    non_null_indices = np.where(~np.isnan(y))[0]

    # Perform polynomial interpolation using scipy's interp1d
    poly_interp = interp1d(non_null_indices, y[non_null_indices], kind='cubic', fill_value="extrapolate")

    # Interpolate the null values
    y_interp = poly_interp(x)

    # # Create a single figure with two subplots
    # plt.figure(figsize=(12, 4))

    # # Plot the original data
    # plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
    # plt.plot(df['time'], y, marker='o', linestyle='-', label='Original Data')
    # plt.title('Original Data: ' + column)
    # plt.xlabel('Time')
    # plt.ylabel(column)
    # plt.grid(True)
    # plt.legend()

    # # Plot the interpolated data
    # plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
    # plt.plot(df['time'], y, 'o', label='Original Data')
    # plt.plot(df['time'], y_interp, '-', label='Interpolated Data')
    # plt.title('Interpolated Data: ' + column)
    # plt.xlabel('Time')
    # plt.ylabel(column)
    # plt.grid(True)
    # plt.legend()

    # # Display the plots
    # plt.tight_layout()
    # plt.show()

    # Update the DataFrame with the interpolated values
    df[column] = y_interp

In [125]:
for column in filtered_intersection_df.columns:
    if filtered_intersection_df[column].isnull().any():
        interpolate(filtered_intersection_df, column)

### Result

In [126]:
print(filtered_intersection_df.isnull().sum())
print(filtered_intersection_df.shape)

time                0
ni_in               0
fe_in               0
cao_in              0
al2o3_in            0
s_m                 0
bc                  0
mc_kilnfeed         0
fc_coal             0
gcv_coal            0
current_pry         0
current_sec1        0
current_sec2        0
current_sec3        0
load                0
power_factor        0
realisasi_beban     0
rpm                 0
kg_tco              0
charge_kiln         0
tdo                 0
pry_p               0
sec_p               0
pry_v               0
sec_v               0
total_coal          0
total_fuel          0
a_f_ratio           0
reductor_consume    0
t_tic162            0
t_tic163            0
metal_temp          0
ni_met              0
c_met               0
si_met              0
fe_met              0
s_met               0
ni_slag             0
fe_slag             0
t_kalsin            0
pic_161             0
loi_kalsin          0
dtype: int64
(2495, 42)


In [127]:
min_max_df = filtered_intersection_df.agg(['min', 'max']).transpose()

# Display the min and max values for each column
print(min_max_df)

                                  min                  max
time              2024-07-31 01:00:00  2024-11-12 07:00:00
ni_in                            1.33                 2.09
fe_in                           11.95                23.62
cao_in                           0.36                  3.5
al2o3_in                          1.0                 4.24
s_m                          1.471361             3.646552
bc                           0.292244             0.694697
mc_kilnfeed                     12.22                29.16
fc_coal                          31.9                 80.0
gcv_coal                       5823.4              6987.85
current_pry                      -0.0               221.97
current_sec1                     -0.0                197.0
current_sec2                     -0.0                229.0
current_sec3                    -14.8                282.2
load                            -22.4                94.82
power_factor                     -1.0                  1

In [128]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [129]:
# Remove outliers from 'total_coal' and 'total_fuel'
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'total_coal')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'total_fuel')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'charge_kiln')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'kg_tco')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'tdo')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_pry')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_sec1')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_sec2')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_sec3')

# Display the shape of the DataFrame after removing outliers
print(filtered_intersection_df.shape)

(1564, 42)


In [130]:
min_max_df = filtered_intersection_df.agg(['min', 'max']).transpose()

# Display the min and max values for each column
print(min_max_df)

                                  min                  max
time              2024-07-31 01:00:00  2024-11-12 07:00:00
ni_in                            1.34                 2.09
fe_in                           11.95                23.62
cao_in                           0.36                  3.5
al2o3_in                          1.0                 3.53
s_m                          1.471361             3.646552
bc                           0.292244             0.694697
mc_kilnfeed                     12.22                29.16
fc_coal                          31.9                 80.0
gcv_coal                       5823.4              6987.85
current_pry                     20.48                23.59
current_sec1                     16.8                 25.8
current_sec2                     17.0                 26.0
current_sec3                     16.9                 25.9
load                            -22.4                94.82
power_factor                     -1.0                 0.

In [131]:
filtered_intersection_df.to_excel('filtered_intersection_df_interpolated.xlsx', index=False)
filtered_intersection_df.to_csv('filtered_intersection_df_interpolated.csv', index=False)

## Model Training

In [103]:
data_df = filtered_intersection_df.copy()
print(data_df.shape)

(1564, 42)


In [104]:
# Initialize the RobustScaler
scaler = RobustScaler()

# Apply Robust Scaling to the DataFrame
scaled_data = scaler.fit_transform(data_df.drop(columns=['time']))

# Convert the scaled data back to a DataFrame
scaled_data_df = pd.DataFrame(scaled_data, columns=data_df.drop(columns=['time']).columns)

In [105]:
input_cols = scaled_data_df.columns[scaled_data_df.columns.get_loc('ni_in'):scaled_data_df.columns.get_loc('t_tic163') + 1]
output_cols = scaled_data_df.columns[scaled_data_df.columns.get_loc('metal_temp'):scaled_data_df.columns.get_loc('loi_kalsin') + 1]

In [106]:
# Define split configurations
split_ratios = [0.1]

# Initialize models
models = {
    'XGBoost': XGBRegressor(),
}

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='uniform_average')  # Average across outputs
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='uniform_average')
    return mse, rmse, mae

# Iterate over split ratios
for test_size in split_ratios:
    print(f"Split ratio: {int((1 - test_size) * 100)}:{int(test_size * 100)}")
    
    # Split the data
    X = scaled_data_df[input_cols]
    y = scaled_data_df[output_cols]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    y_train = y_train.values
    y_test = y_test.values

    # Initialize results for this split ratio
    results_mse, results_rmse, results_mae = [], [], []
    
    # Train each model
    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Inverse transform y_test and y_pred to original scale
        y_test_original = scaler.inverse_transform(np.hstack([X_test, y_test]))[:, len(input_cols):]
        y_pred_original = scaler.inverse_transform(np.hstack([X_test, y_pred]))[:, len(input_cols):]
        
        # Calculate metrics for each target
        for col_idx, col_name in enumerate(output_cols):
            mse = mean_squared_error(y_test_original[:, col_idx], y_pred_original[:, col_idx])
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test_original[:, col_idx], y_pred_original[:, col_idx])
            
            # Append results
            results_mse.append([name, col_name, mse])
            results_rmse.append([name, col_name, rmse])
            results_mae.append([name, col_name, mae])

        if name == 'XGBoost':
            # Save the XGBoost model in pkl format
            with open(f'xgboost_model_{int((1 - test_size) * 100)}_{int(test_size * 100)}.pkl', 'wb') as file:
                pickle.dump(model, file)
    
    # Create DataFrames for metrics
    mse_df = pd.DataFrame(results_mse, columns=['Model', 'Feature', 'MSE'])
    rmse_df = pd.DataFrame(results_rmse, columns=['Model', 'Feature', 'RMSE'])
    mae_df = pd.DataFrame(results_mae, columns=['Model', 'Feature', 'MAE'])

    # Pivot DataFrames
    mse_pivot = mse_df.pivot(index='Feature', columns='Model', values='MSE')
    rmse_pivot = rmse_df.pivot(index='Feature', columns='Model', values='RMSE')
    mae_pivot = mae_df.pivot(index='Feature', columns='Model', values='MAE')

    # Display results
    print("MSE Table:")
    print(mse_pivot)
    print("\nRMSE Table:")
    print(rmse_pivot)
    print("\nMAE Table:")
    print(mae_pivot)

    # Save results for this split ratio
    filename = f'xgb_model_performance_metrics_{int((1 - test_size) * 100)}_{int(test_size * 100)}.xlsx'
    with pd.ExcelWriter(filename) as writer:
        mse_pivot.to_excel(writer, sheet_name='MSE')
        rmse_pivot.to_excel(writer, sheet_name='RMSE')
        mae_pivot.to_excel(writer, sheet_name='MAE')

Split ratio: 90:10
Training XGBoost...
MSE Table:
Model           XGBoost
Feature                
c_met          0.003208
fe_met         0.022630
fe_slag        0.143248
loi_kalsin     0.032028
metal_temp    67.789136
ni_met         0.060521
ni_slag        0.000021
pic_161        1.800694
s_met          0.006027
si_met         0.002547
t_kalsin    1440.409318

RMSE Table:
Model         XGBoost
Feature              
c_met        0.056643
fe_met       0.150432
fe_slag      0.378482
loi_kalsin   0.178964
metal_temp   8.233416
ni_met       0.246010
ni_slag      0.004550
pic_161      1.341899
s_met        0.077636
si_met       0.050467
t_kalsin    37.952725

MAE Table:
Model         XGBoost
Feature              
c_met        0.040135
fe_met       0.105521
fe_slag      0.255039
loi_kalsin   0.105510
metal_temp   5.384298
ni_met       0.130421
ni_slag      0.003283
pic_161      0.799523
s_met        0.033909
si_met       0.035015
t_kalsin    27.485651


## Optimization

In [107]:
# Load the XGBoost model from pkl format
with open('xgboost_model_90_10.pkl', 'rb') as file:
    xgboost_model = pickle.load(file)

In [108]:
# Load the CSV file into a DataFrame
df = filtered_intersection_df.copy()
df = df.drop(columns=['time'])

fixed_input_cols = ['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'mc_kilnfeed', 'fc_coal', 'gcv_coal', 'kg_tco', 'charge_kiln', 'tdo']

In [109]:
print("fixed input cols")
print(fixed_input_cols)
print("input cols")
print(input_cols)
print("output cols")
print(output_cols)

fixed input cols
['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'mc_kilnfeed', 'fc_coal', 'gcv_coal', 'kg_tco', 'charge_kiln', 'tdo']
input cols
Index(['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'mc_kilnfeed',
       'fc_coal', 'gcv_coal', 'current_pry', 'current_sec1', 'current_sec2',
       'current_sec3', 'load', 'power_factor', 'realisasi_beban', 'rpm',
       'kg_tco', 'charge_kiln', 'tdo', 'pry_p', 'sec_p', 'pry_v', 'sec_v',
       'total_coal', 'total_fuel', 'a_f_ratio', 'reductor_consume', 't_tic162',
       't_tic163'],
      dtype='object')
output cols
Index(['metal_temp', 'ni_met', 'c_met', 'si_met', 'fe_met', 's_met', 'ni_slag',
       'fe_slag', 't_kalsin', 'pic_161', 'loi_kalsin'],
      dtype='object')


In [110]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from sklearn.preprocessing import RobustScaler
import pandas as pd
import numpy as np

class FeNi(ElementwiseProblem):
    def __init__(self, xgboost_model, current_condition, scaler_X, scaler_y, input_cols, fixed_input_cols, output_cols, df):
        self.xgboost_model = xgboost_model
        self.current_condition = current_condition
        self.scaler_X = scaler_X
        self.scaler_y = scaler_y
        self.input_cols = input_cols
        self.fixed_input_cols = fixed_input_cols
        self.output_cols = output_cols
        self.df = df  # Store dataset for min value calculations

        # Get min values for constrained columns
        self.min_values = df.min()

        n_var = len(input_cols) - len(fixed_input_cols)
        super().__init__(n_var=n_var, n_obj=2, n_constr=15, xl=0, xu=1)  # Updated constraints count

    def _evaluate(self, x, out, *args, **kwargs):
        condition = self.current_condition.copy()

        # Validate x size
        required_vars = len(self.input_cols) - len(self.fixed_input_cols)
        if len(x) != required_vars:
            raise ValueError(f"Expected {required_vars} variables, but got {len(x)}")

        # Update condition with optimized variables
        var_index = 0
        for col in self.input_cols:
            if col not in self.fixed_input_cols:
                condition[col] = x[var_index]
                var_index += 1

        # Set min value for constrained columns
        constraint_checks = {
            'metal_temp': (1300, None),
            'ni_met': (17, None),
            'c_met': (1, 3),
            'si_met': (1, 2),
            'fe_met': (0, None),
            's_met': (None, 0.4),
            'ni_slag': (0, None),
            'fe_slag': (0, None),
            't_kalsin': (600, None),
            'pic_161': (0, None),
            'loi_kalsin': (None, 1),
        }

        for col, (lower, upper) in constraint_checks.items():
            if lower is not None and lower <= 0:
                condition[col] = self.min_values[col]

        # Ensure DataFrame format before transformation
        condition_scaled = self.scaler_X.transform(pd.DataFrame([condition[self.input_cols]], columns=self.input_cols))
        predicted_output_scaled = self.xgboost_model.predict(condition_scaled)
        predicted_output = self.scaler_y.inverse_transform(predicted_output_scaled)

        # Objective: minimize total_coal and total_fuel
        total_coal = condition['total_coal']
        total_fuel = condition['total_fuel']
        out["F"] = [total_coal, total_fuel]

        # Constraints
        constraints = []
        output_values = predicted_output[0]

        for col, (lower, upper) in constraint_checks.items():
            value = output_values[self.output_cols.get_loc(col)]
            if lower is not None:
                constraints.append(value - lower)
            if upper is not None:
                constraints.append(upper - value)

        # Ensure total_coal and total_fuel are at least the dataset minimum
        constraints.append(self.min_values["total_coal"] - total_coal)
        constraints.append(self.min_values["total_fuel"] - total_fuel)

        out["G"] = constraints

# Load and preprocess data
X = df[input_cols]
y = df[output_cols]
scaler_X = RobustScaler()
scaler_y = RobustScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
current_condition = df.sample(n=1).iloc[0]

# Define the optimization problem
problem = FeNi(xgboost_model, current_condition, scaler_X, scaler_y, input_cols, fixed_input_cols, output_cols, df)

# Run NSGA2 genetic algorithm with history tracking
algorithm = NSGA2(pop_size=100)
res = minimize(problem, algorithm, termination=('n_gen', 200), seed=1, verbose=True, save_history=True)

# Check if optimization returned a valid result
optimized_values = None
if res.X is None:
    print("Optimization failed or returned no valid result. Selecting fittest individual from last population...")
    
    last_population = None
    last_fitness = None
    
    if res.pop is not None:
        last_population = np.array([ind.X for ind in res.pop])
        last_fitness = np.array([ind.F for ind in res.pop])
    elif res.history and len(res.history) > 0:
        last_population = np.array([ind.X for ind in res.history[-1].pop])
        last_fitness = np.array([ind.F for ind in res.history[-1].pop])
    
    if last_population is not None:
        best_idx = np.argmin(last_fitness.sum(axis=1))
        optimized_values = last_population[best_idx]

# If we found an optimized solution, process it
if optimized_values is not None:
    optimized_condition = current_condition.copy()
    var_index = 0
    for col in input_cols:
        if col not in fixed_input_cols:
            optimized_condition[col] = optimized_values[var_index]
            var_index += 1

    # Ensure DataFrame format before transformation
    optimized_condition_scaled = scaler_X.transform(pd.DataFrame([optimized_condition[input_cols]], columns=input_cols))
    optimized_output_scaled = xgboost_model.predict(optimized_condition_scaled)
    optimized_output = scaler_y.inverse_transform(optimized_output_scaled)

    for i, col in enumerate(output_cols):
        optimized_condition[col] = optimized_output[0][i]

    print("\nOptimized Row:")
    print(optimized_condition[input_cols.tolist() + output_cols.tolist()])

    # Check for values outside expected range
    min_values = df.min()
    max_values = df.max()
    print("\nColumns outside the min-max range:")
    for col in input_cols.tolist() + output_cols.tolist():
        if optimized_condition[col] < min_values[col] or optimized_condition[col] > max_values[col]:
            print(f"{col}: {optimized_condition[col]} (Min: {min_values[col]}, Max: {max_values[col]})")
else:
    print("No valid optimized solution found.")

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  4.277824E+03 |  4.326762E+03 |             - |             -
     2 |      200 |      1 |  4.277660E+03 |  4.303131E+03 |             - |             -
     3 |      300 |      1 |  4.277505E+03 |  4.294420E+03 |             - |             -


KeyboardInterrupt: 

In [ ]:
print(input_cols)

Index(['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'mc_kilnfeed',
       'fc_coal', 'gcv_coal', 'current_pry', 'current_sec1', 'current_sec2',
       'current_sec3', 'load', 'power_factor', 'realisasi_beban', 'rpm',
       'kg_tco', 'charge_kiln', 'tdo', 'pry_p', 'sec_p', 'pry_v', 'sec_v',
       'total_coal', 'total_fuel', 'a_f_ratio', 'reductor_consume', 't_tic162',
       't_tic163'],
      dtype='object')


In [ ]:
print(df.columns)

Index(['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'mc_kilnfeed',
       'fc_coal', 'gcv_coal', 'current_pry', 'current_sec1', 'current_sec2',
       'current_sec3', 'load', 'power_factor', 'realisasi_beban', 'rpm',
       'kg_tco', 'charge_kiln', 'tdo', 'pry_p', 'sec_p', 'pry_v', 'sec_v',
       'total_coal', 'total_fuel', 'a_f_ratio', 'reductor_consume', 't_tic162',
       't_tic163', 'metal_temp', 'ni_met', 'c_met', 'si_met', 'fe_met',
       's_met', 'ni_slag', 'fe_slag', 't_kalsin', 'pic_161', 'loi_kalsin'],
      dtype='object')
